In [ ]:
import os
import cv2
import math
import glob
import numpy as np
from utils import *
from fingerprint_enhancer import enhance_fingerprint

RAW_INPUT_FOLDERS = ["Images/Genuine/", "Images/Impostor/"]
ENHANCED_FOLDERS = ["Images/Genuine_Enhanced/", "Images/Impostor_Enhanced/"]
THINNED_FOLDERS = ["Images/Genuine_Thinned/", "Images/Impostor_Thinned/"]
MINUTIAE_FOLDERS = ["Images/Genuine_Minutiae/", "Images/Impostor_Minutiae/"]
IMAGE_EXTENSIONS = ['png', 'jpg','bmp']

def enhance_fingerprint_image(image):
    enhanced_image = enhance_fingerprint(image)
    if enhanced_image.dtype == np.bool_:
        enhanced_image = (enhanced_image * 255).astype(np.uint8)
    return enhanced_image

def process_and_save_enhanced_fingerprints(input_folder, output_folder):
    if not os.path.exists(input_folder):
        print(f"Error: Input folder '{input_folder}' does not exist.")
        return
    os.makedirs(output_folder, exist_ok=True)
    for filename in os.listdir(input_folder):
        if not filename.lower().endswith(tuple(IMAGE_EXTENSIONS)):
            continue
        imgpath = os.path.join(input_folder, filename)
        image = cv2.imread(imgpath, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Error: Failed to read image {imgpath}.")
            continue
        enhanced_image = enhance_fingerprint_image(image)
        output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.jpg")
        cv2.imwrite(output_path, enhanced_image)

for in_folder, out_folder in zip(RAW_INPUT_FOLDERS, ENHANCED_FOLDERS):
    process_and_save_enhanced_fingerprints(in_folder, out_folder)

def process_image(image):
    gx, gy = cv.Sobel(image, cv.CV_32F, 1, 0), cv.Sobel(image, cv.CV_32F, 0, 1)
    gx2, gy2 = gx**2, gy**2
    gm = np.sqrt(gx2 + gy2)

    sum_gm = cv.boxFilter(gm, -1, (25, 25), normalize=False)
    thr = sum_gm.max() * 0.2
    mask = cv.threshold(sum_gm, thr, 255, cv.THRESH_BINARY)[1].astype(np.uint8)

    W = (23, 23)
    gxx = cv.boxFilter(gx2, -1, W, normalize=False)
    gyy = cv.boxFilter(gy2, -1, W, normalize=False)
    gxy = cv.boxFilter(gx * gy, -1, W, normalize=False)
    gxx_gyy = gxx - gyy
    gxy2 = 2 * gxy
    orientations = (cv.phase(gxx_gyy, -gxy2) + np.pi) / 2

    h, w = image.shape
    y_start, y_end = max(0, h//2-40), min(h, h//2+40)
    x_start, x_end = max(0, w//2-25), min(w, w//2+25)
    region = image[y_start:y_end, x_start:x_end]

    smoothed = cv.blur(region, (5,5))
    xs = np.sum(smoothed, 1)
    local_maxima = np.nonzero(np.r_[False, xs[1:] > xs[:-1]] & np.r_[xs[:-1] >= xs[1:], False])[0]
    ridge_period = np.average(local_maxima[1:] - local_maxima[:-1]) if len(local_maxima) > 1 else 10

    or_count = 8
    gabor_bank = [gabor_kernel(ridge_period, o) for o in np.arange(0, np.pi, np.pi/or_count)]
    nf = 255 - image
    all_filtered = np.array([cv.filter2D(nf, cv.CV_32F, f) for f in gabor_bank])

    y_coords, x_coords = np.indices(image.shape)
    orientation_idx = np.round(((orientations % np.pi) / np.pi) * or_count).astype(np.int32) % or_count
    filtered = all_filtered[orientation_idx, y_coords, x_coords]

    enhanced = mask & np.clip(filtered, 0, 255).astype(np.uint8)
    _, ridge_lines = cv.threshold(enhanced, 32, 255, cv.THRESH_BINARY)
    return cv.ximgproc.thinning(ridge_lines, thinningType=cv.ximgproc.THINNING_GUOHALL)

for folder in THINNED_FOLDERS:
    os.makedirs(folder, exist_ok=True)

for in_folder, out_folder in zip(ENHANCED_FOLDERS, THINNED_FOLDERS):
    for ext in IMAGE_EXTENSIONS:
        pattern = os.path.join(in_folder, f'*.{ext}')
        for file_path in glob.glob(pattern):
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                skeleton = process_image(img)
                skeleton_8bit = 255 - skeleton  # Invert if needed
                filename = os.path.splitext(os.path.basename(file_path))[0] + '.jpg'
                output_path = os.path.join(out_folder, filename)
                cv2.imwrite(output_path, skeleton_8bit)

for out_folder in MINUTIAE_FOLDERS:
    os.makedirs(out_folder, exist_ok=True)
    os.makedirs(os.path.join(out_folder, 'Fingerprint+Minutiae'), exist_ok=True)
    os.makedirs(os.path.join(out_folder, 'Minutiae'), exist_ok=True)

def draw_minutiae(fingerprint, minutiae, termination_color=(255, 0, 0), bifurcation_color=(0, 0, 255)):
    res = cv2.cvtColor(fingerprint, cv2.COLOR_GRAY2BGR)
    for x, y, t, *d in minutiae:
        color = termination_color if t else bifurcation_color
        if len(d) == 0:
            cv2.drawMarker(res, (x, y), color, cv2.MARKER_CROSS, 8)
        else:
            d = d[0]
            ox = int(round(math.cos(d) * 7))
            oy = int(round(math.sin(d) * 7))
            if 0 <= x + ox < res.shape[1] and 0 <= y - oy < res.shape[0]:
                cv2.circle(res, (x, y), 3, color, 1, cv2.LINE_AA)
                cv2.line(res, (x, y), (x + ox, y - oy), color, 1, cv2.LINE_AA)
    return res

def compute_crossing_number(values):
    return np.count_nonzero(values < np.roll(values, -1))

def angle_abs_difference(a, b):
    return math.pi - abs(abs(a - b) - math.pi)

def angle_mean(a, b):
    return math.atan2((math.sin(a) + math.sin(b)) / 2, ((math.cos(a) + math.cos(b)) / 2))

for in_folder, out_folder in zip(THINNED_FOLDERS, MINUTIAE_FOLDERS):
    fingerprint_minutiae = os.path.join(out_folder, 'Fingerprint+Minutiae')
    remove_fingerprint_minutiae = os.path.join(out_folder, 'Minutiae')
    for filename in os.listdir(in_folder):
        if filename.lower().endswith(tuple(IMAGE_EXTENSIONS)):
            try:
                fingerprint = cv2.imread(os.path.join(in_folder, filename), cv2.IMREAD_GRAYSCALE)
                if fingerprint is None:
                    print(f"Warning: Could not read image {filename}. Skipping.")
                    continue
                gx = cv2.Sobel(fingerprint, cv2.CV_32F, 1, 0)
                gy = cv2.Sobel(fingerprint, cv2.CV_32F, 0, 1)
                gx2, gy2 = gx**2, gy**2
                gm = np.sqrt(gx2 + gy2)
                sum_gm = cv2.boxFilter(gm, -1, (25, 25), normalize=False)
                thr = sum_gm.max() * 0.2
                mask = cv2.threshold(sum_gm, thr, 255, cv2.THRESH_BINARY)[1].astype(np.uint8)
                W = (23, 23)
                gxx = cv2.boxFilter(gx2, -1, W, normalize=False)
                gyy = cv2.boxFilter(gy2, -1, W, normalize=False)
                gxy = cv2.boxFilter(gx * gy, -1, W, normalize=False)
                gxx_gyy = gxx - gyy
                gxy2 = 2 * gxy
                orientations = (cv2.phase(gxx_gyy, -gxy2) + np.pi) / 2
                region = fingerprint[10:90,80:130]
                smoothed = cv2.blur(region, (5,5), -1)
                xs = np.sum(smoothed, 1)
                local_maxima = np.nonzero(np.r_[False, xs[1:] > xs[:-1]] & np.r_[xs[:-1] >= xs[1:], False])[0]
                if len(local_maxima) < 2:
                    print(f"Warning: Insufficient local maxima found for {filename}. Setting default ridge_period.")
                    ridge_period = 10
                else:
                    distances = local_maxima[1:] - local_maxima[:-1]
                    distances = distances[distances > 0]
                    if distances.size == 0 or np.isnan(distances).any():
                        print(f"Warning: distances array is empty or contains NaN values for {filename}. Setting default ridge_period.")
                        ridge_period = 10
                    else:
                        ridge_period = np.average(distances)
                or_count = 8
                gabor_bank = [gabor_kernel(ridge_period, o) for o in np.arange(0, np.pi, np.pi/or_count)]
                nf = 255-fingerprint
                all_filtered = np.array([cv2.filter2D(nf, cv2.CV_32F, f) for f in gabor_bank])
                y_coords, x_coords = np.indices(fingerprint.shape)
                orientation_idx = np.round(((orientations % np.pi) / np.pi) * or_count).astype(np.int32) % or_count
                filtered = all_filtered[orientation_idx, y_coords, x_coords]
                enhanced = mask & np.clip(filtered, 0, 255).astype(np.uint8)
                _, ridge_lines = cv2.threshold(enhanced, 32, 255, cv2.THRESH_BINARY)
                skeleton = cv2.ximgproc.thinning(ridge_lines, thinningType=cv2.ximgproc.THINNING_GUOHALL)
                filename_base, _ = os.path.splitext(filename)
                white_image = np.full_like(fingerprint, 255, dtype=np.uint8)
                cn_filter = np.array([[  1,  2,  4],
                                      [128,  0,  8],
                                      [ 64, 32, 16]
                                    ])
                all_8_neighborhoods = [np.array([int(d) for d in f'{x:08b}'])[::-1] for x in range(256)]
                cn_lut = np.array([compute_crossing_number(x) for x in all_8_neighborhoods]).astype(np.uint8)
                skeleton01 = np.where(skeleton!=0, 1, 0).astype(np.uint8)
                neighborhood_values = cv2.filter2D(skeleton01, -1, cn_filter, borderType=cv2.BORDER_CONSTANT)
                cn = cv2.LUT(neighborhood_values, cn_lut)
                cn[skeleton==0] = 0
                minutiae = [(x,y,cn[y,x]==1) for y, x in zip(*np.where(np.isin(cn, [1,3])))]
                mask_distance = cv2.distanceTransform(cv2.copyMakeBorder(mask, 1, 1, 1, 1, cv2.BORDER_CONSTANT), cv2.DIST_C, 3)[1:-1,1:-1]
                filtered_minutiae = list(filter(lambda m: mask_distance[m[1], m[0]]>10, minutiae))
                def compute_next_ridge_following_directions(previous_direction, values):
                    next_positions = np.argwhere(values!=0).ravel().tolist()
                    if len(next_positions) > 0 and previous_direction != 8:
                        next_positions.sort(key = lambda d: 4 - abs(abs(d - previous_direction) - 4))
                        if next_positions[-1] == (previous_direction + 4) % 8:
                            next_positions = next_positions[:-1]
                    return next_positions
                r2 = 2**0.5
                xy_steps = [(-1,-1,r2),( 0,-1,1),( 1,-1,r2),( 1, 0,1),( 1, 1,r2),( 0, 1,1),(-1, 1,r2),(-1, 0,1)]
                nd_lut = [[compute_next_ridge_following_directions(pd, x) for pd in range(9)] for x in all_8_neighborhoods]
                def follow_ridge_and_compute_angle(x, y, d = 8):
                    px, py = x, y
                    length = 0.0
                    while length < 20:
                        next_directions = nd_lut[neighborhood_values[py,px]][d]
                        if len(next_directions) == 0:
                            break
                        if (any(cn[py + xy_steps[nd][1], px + xy_steps[nd][0]] != 2 for nd in next_directions)):
                            break
                        d = next_directions[0]
                        ox, oy, l = xy_steps[d]
                        px += ox ; py += oy ; length += l
                    return math.atan2(-py+y, px-x) if length >= 10 else None
                valid_minutiae = []
                for x, y, term in filtered_minutiae:
                    d = None
                    if term:
                        d = follow_ridge_and_compute_angle(x, y)
                    else:
                        dirs = nd_lut[neighborhood_values[y,x]][8]
                        if len(dirs)==3:
                            angles = [follow_ridge_and_compute_angle(x+xy_steps[d][0], y+xy_steps[d][1], d) for d in dirs]
                            if all(a is not None for a in angles):
                                a1, a2 = min(((angles[i], angles[(i+1)%3]) for i in range(3)), key=lambda t: angle_abs_difference(t[0], t[1]))
                                d = angle_mean(a1, a2)
                    if d is not None:
                        valid_minutiae.append( (x, y, term, d) )
                minutiae_skeleton = draw_minutiae(white_image, valid_minutiae)
                minutiae_fingerprint = draw_minutiae(fingerprint, valid_minutiae)
                cv2.imwrite(os.path.join(fingerprint_minutiae, f'{filename_base}.jpg'), minutiae_fingerprint)
                remove_fingerprint = draw_minutiae(white_image, valid_minutiae)
                cv2.imwrite(os.path.join(remove_fingerprint_minutiae, f'{filename_base}.jpg'), remove_fingerprint)
                print(f"Processed: {filename}")
            except Exception as e:
                print(f"Error processing {filename}: {e}")

print("\n*** Processed Completed ***")